## 0. Importing Libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.autograd import Variable
import torch.optim as optim


In [10]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = CIFAR10(root='./data', train=True, download=True, transform=transform)

test_data = CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

100%|██████████| 170498071/170498071 [02:33<00:00, 1113433.92it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
